<a href="https://colab.research.google.com/github/rodzhttp/Pyspark1/blob/main/pyspark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**< Pyspark Project >**

In [ ]:
# Instala as bibliotecas necessárias
!pip install pyspark findspark

# Configura o ambiente para encontrar o Spark
import findspark
from google.colab import drive
findspark.init()

# Inicia a "Sessão Spark", que é o nosso motor para processar os dados
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("AnaliseVoos").getOrCreate()

print("✅ Sessão Spark iniciada com sucesso!")

✅ Sessão Spark iniciada com sucesso!


In [36]:
# O caminho para a pasta que contém seus arquivos
caminho_base = '/content/drive/My Drive/Colab Notebooks/Pyspark project/'

# Caminho completo para cada arquivo
caminho_flights = caminho_base + 'flights.csv'
caminho_airlines = caminho_base + 'airlines.csv'
caminho_airports = caminho_base + 'airports.csv'

# Lendo os arquivos para DataFrames
df_flights = spark.read.csv(caminho_flights, header=True, inferSchema=True)
df_airlines = spark.read.csv(caminho_airlines, header=True, inferSchema=True)
df_airports = spark.read.csv(caminho_airports, header=True, inferSchema=True)

display(df_flights.show(5))

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

None

In [38]:
from pyspark.sql.functions import avg, desc, round

# Esta primeira parte está correta
media_atrasos = df_flights.groupBy("AIRLINE") \
    .agg(
        round(avg("ARRIVAL_DELAY"), 2).alias("media_atraso_chegada")
    )

# O join também está correto
resultado_atrasos = media_atrasos.join(
        df_airlines,
        media_atrasos.AIRLINE == df_airlines.IATA_CODE
    ) \
    .orderBy(desc("media_atraso_chegada"))

# --- CORREÇÃO AQUI ---
# No select, especificamos que queremos a coluna "AIRLINE" que veio de df_airlines
# e damos a ela um novo nome ('alias') para o resultado ficar mais claro.
print("Média de Atrasos na Chegada (em minutos) por Companhia Aérea:")
resultado_atrasos.select(
    df_airlines.AIRLINE.alias("NOME_DA_COMPANHIA"),
    "media_atraso_chegada"
).show()

Média de Atrasos na Chegada (em minutos) por Companhia Aérea:
+--------------------+--------------------+
|   NOME_DA_COMPANHIA|media_atraso_chegada|
+--------------------+--------------------+
|    Spirit Air Lines|               14.47|
|Frontier Airlines...|                12.5|
|     JetBlue Airways|                6.68|
|Atlantic Southeas...|                6.59|
|American Eagle Ai...|                6.46|
|Skywest Airlines ...|                5.85|
|United Air Lines ...|                5.43|
|      Virgin America|                4.74|
|Southwest Airline...|                4.37|
|     US Airways Inc.|                3.71|
|American Airlines...|                3.45|
|Hawaiian Airlines...|                2.02|
|Delta Air Lines Inc.|                0.19|
|Alaska Airlines Inc.|               -0.98|
+--------------------+--------------------+

